<a href="https://colab.research.google.com/github/hudengjunai/Colab_notebooks/blob/master/tensorrt_speedup_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip list

#  so the conclusion is tensorrt will speedup 2x in mobilenet

In [0]:
import tensorflow.contrib.tensorrt as trt

In [0]:
!wget --quiet https://raw.githubusercontent.com/Tony607/tf_jetson_nano/master/data/elephant.jpg

In [18]:
!ls #sample_data/

elephant.jpg  model  sample_data


In [0]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as Net
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input,decode_predictions

In [0]:
import numpy as np
import os

img_path = './elephant.jpg'
model_path = './model'


In [21]:
if not os.path.exists(model_path):
  os.mkdir(model_path)
  
model_fname = os.path.join(model_path,'model.h5')

os.makedirs(model_path,exist_ok=True)

img_height = 224
img_width = 224

model = Net(weights='imagenet')

img = image.load_img(img_path,target_size=(img_height,img_width))
x =image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print("predicted si ",decode_predictions(preds,top=3)[0])

model.save(model_fname)

predicted si  [('n02504458', 'African_elephant', 0.705134), ('n01871265', 'tusker', 0.1551433), ('n02504013', 'Indian_elephant', 0.039011613)]


In [13]:
## Benckmark keras prediction speed
import time
times= []
for i in range(30):
  start_time =time.time()
  preds = model.predict(x)
  delta = time.time() - start_time
  times.append(delta)
  
  
mean_delta = np.array(times).mean()
fps = 1/mean_delta
print("average(sec):{0},fps:{1}".format(mean_delta,fps))

tf.keras.backend.clear_session()


average(sec):0.021813845634460448,fps:45.84244414108482


In [23]:
!ls model


model.h5


## Freeze graph and export .pb file

In [0]:
import tensorflow as tf
from tensorflow.python.framework import graph_io

from tensorflow.keras.models import load_model

tf.keras.backend.clear_session()


In [0]:
save_pb_dir = './model'
model_name = './model/model.h5'

def freeze_graph(graph,\
                 session,\
                 output,\
                 save_pb_dir='.',\
                 save_pb_name='frozon_model.pb',\
                 save_pb_as_text=False):
  with graph.as_default():
    #get the inference graph
    graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
    graphdef_frozen = tf.graph_util.convert_variables_to_constants(session,graphdef_inf,output)
    graph_io.write_graph(graphdef_frozen,save_pb_dir,save_pb_name,as_text=save_pb_as_text)
    return graphdef_frozen
    

In [27]:
tf.keras.backend.set_learning_phase(0)

model = load_model(model_fname)

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

print("input names \n",input_names)
print("output names \n",output_names)


W0815 05:55:20.357565 140715131443072 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 05:55:20.362101 140715131443072 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 05:55:20.367070 140715131443072 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecat

input names 
 ['input_1']
output names 
 ['Logits/Softmax']


In [0]:
tf.keras.backend.set_learning_phase?


In [0]:
frozen_graph = freeze_graph(session.graph,session,\
                            [out.op.name for out in model.outputs],\
                            save_pb_dir=save_pb_dir)

In [0]:

import tensorflow.contrib.tensorrt as trt

In [0]:
#Optimize with TensorRt

### Optimize with TensorRt

In [52]:
trt_graph = trt.create_inference_graph(input_graph_def = frozen_graph,\
                                      outputs = output_names,\
                                      max_batch_size = 1,\
                                      max_workspace_size_bytes = 1 << 25,\
                                      minimum_segment_size = 50)
graph_io.write_graph(trt_graph,'./model','trt_graph.pb',as_text=False)

'./model/trt_graph.pb'

In [53]:
!ls model

frozon_model.pb  model.h5  trt_graph.pb


In [0]:
tf.reset_default_graph()
def get_frozen_graph(graph_file):
  with tf.gfile.FastGFile(graph_file,'rb') as f:
    graphdef = tf.GraphDef()
    graphdef.ParseFromString(f.read())
    return graphdef
  

In [0]:
graphdef = get_frozen_graph('./model/trt_graph.pb')

In [0]:
sess = tf.Session()
tf.import_graph_def(graphdef,name='')
graph = tf.get_default_graph()

In [0]:
input_node = graph.get_tensor_by_name('input_1:0')

In [0]:
output_node = graph.get_tensor_by_name('Logits/Softmax:0')#len(graphdef.node)

In [0]:
img_path = './elephant.jpg'
img = image.load_img(img_path,target_size=[224,224])
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

feed_dict = {input_node:x}
preds = sess.run(output_node,feed_dict=feed_dict)

In [87]:
times = []
for i in range(30):
  start_time = time.time()
  preds = sess.run(output_node,feed_dict=feed_dict)
  delta = time.time() - start_time
  times.append(delta)
mean_time = np.array(times).mean()
fps = 1/mean_time
print("mean time used is {0},fps:{1} ".format(mean_time,fps))

mean time used is 0.010166740417480469,fps:98.359942217136 


In [0]:
# so the conclusion is tensorrt will speedup 2x in mobilenet